## HRA Hierarchical Tissue Unit Annotation

In this notebook, we will build on [an existing one on hierarchical tissue unit annotation](https://github.com/HickeyLab/Hierarchical-Tissue-Unit-Annotation) by [Dr. John Hickey](https://bme.duke.edu/people/john-hickey/). Outputs from this workbook are featured in Fig. 3 in the paper titled "Human BioMolecular Atlas Program (HuBMAP): 3D Human Reference Atlas Construction and Usage" (accepted for publication in Nature Methods, see preprint [on bioRxiv](https://doi.org/10.1101/2024.03.27.587041)). In this notebook, we will create a Vasculature Common Coordinate Framework (VCCF) distance visualization for the same donor and region (B006, proximal jejunum) as in the paper, see below.

<img src="images/fig_3_original.jpg" width="400">


Concretely, we will take the original CSV file from Dryad with cell positions, types, donor IDs, and extraction sites, and then create a node-dist-vis widget plus a Cell Distance Explorer widget. For more information and documentation on hra-jupyter-widgets, please see [https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb](https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb).

## Load libraries

In [1]:
# Import native packages
import os
from pprint import pprint
import csv

In [2]:
#Install and import external packages
%pip install matplotlib pandas ipywidgets hra_jupyter_widgets

import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets

# Import hra-jupyter-widgets. For documentation, please see https://github.com/x-atlas-consortia/hra-jupyter-widgets/blob/main/usage.ipynb
from hra_jupyter_widgets import (
    BodyUi,
    CdeVisualization, # in this example, we will use this one...
    Eui,
    EuiOrganInformation,
    FtuExplorer,
    FtuExplorerSmall,
    MedicalIllustration,
    ModelViewer,
    NodeDistVis, # ...and this one, but all of them are usable for different purposes!
    Rui,
)

Note: you may need to restart the kernel to use updated packages.


## Download data from Dryad

Let's get the data from Dryad. Note that this is a ~3GB file. We will use `curl` to dowload it.

In [3]:
# Make sure the data folder is present
folder_path = "data/"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

# Define the path to the file. 
file_path = f'{folder_path}/23_09_CODEX_HuBMAP_alldata_Dryad_merged.csv'

# Check if the file exists
if not os.path.exists(file_path):
    # If the file doesn't exist, run the curl command
    !curl -L https://datadryad.org/api/v2/files/2572152/download -o {file_path}
    print(f"File downloaded and saved at {file_path}")
else:
    print(f"File already exists at {file_path}")

Folder 'data/' already exists.
File already exists at data//23_09_CODEX_HuBMAP_alldata_Dryad_merged.csv


## Read data as DataFrame

In [4]:
# Read the CSV file and convert it to a df
df = pd.read_csv(file_path, index_col=0)
df

,MUC2,SOX9,MUC1,CD31,Synapto,CD49f,CD15,CHGA,CDX2,ITLN1,...,Cell Type em,Cell subtype,Neighborhood,Neigh_sub,Neighborhood_Ind,NeighInd_sub,Community,Major Community,Tissue Segment,Tissue Unit
0,-0.303994,-0.163727,-0.587608,-0.212903,0.164173,-0.664863,0.049305,0.003616,-0.377532,-0.450794,...,NK,Immune,Mature Epithelial,Epithelial,Mature Epithelial,Epithelial,Plasma Cell Enriched,Immune,Mucosa,Mucosa
1,-0.301927,-0.491706,-0.500804,-0.243205,-0.142568,-0.664861,-0.182627,-0.117573,-0.182754,-0.236199,...,NK,Immune,Transit Amplifying Zone,Epithelial,Mature Epithelial,Epithelial,Mature Epithelial,Epithelial,Mucosa,Mucosa
2,-0.302206,-0.547234,-0.510705,-0.235309,-0.217185,-0.622758,-0.296486,-0.091504,-0.268055,-0.355383,...,NK,Immune,Innate Immune Enriched,Immune,Innate Immune Enriched,Immune,Innate Immune Enriched,Immune,Mucosa,Mucosa
3,-0.304219,-0.613068,-0.584499,-0.243757,-0.266696,-0.658449,-0.299027,-0.121460,-0.345381,-0.450792,...,NK,Immune,Stroma & Innate Immune,Stromal,Stroma & Innate Immune,Stromal,Stroma,Stroma,Subucosa,Submucosa
4,-0.294644,-0.615593,-0.570580,-0.247548,-0.042246,-0.642230,-0.299031,-0.121458,-0.377533,-0.450797,...,NK,Immune,Outer Follicle,Immune,Outer Follicle,Immune,Follicle,Immune,Mucosa,Mucosa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2603212,0.351916,0.693827,-0.081489,-0.240643,0.008875,0.143445,0.373710,-0.097896,0.869830,0.579653,...,CD66+ Enterocyte,Epithelial,CD66+ Mature Epithelial,Epithelial,CD66+ Mature Epithelial,Epithelial,Secretory Epithelial,Epithelial,Mucosa,Mucosa
2603213,0.233642,0.171892,0.141842,-0.236145,-0.097772,-0.099283,0.626185,-0.105545,0.092076,0.682969,...,CD66+ Enterocyte,Epithelial,CD66+ Mature Epithelial,Epithelial,CD66+ Mature Epithelial,Epithelial,Secretory Epithelial,Epithelial,Mucosa,Mucosa
2603214,-0.212237,-0.280904,-0.197833,-0.245638,-0.152563,-0.125035,0.430416,-0.105787,-0.038327,-0.173319,...,CD66+ Enterocyte,Epithelial,CD8+ T Enriched IEL,Immune,CD8+ T Enriched IEL,Immune,Mature Epithelial,Epithelial,Mucosa,Mucosa
2603215,-0.328666,0.607609,-0.180362,-0.247351,-0.143742,-0.169576,1.095596,-0.113879,0.370160,-0.133272,...,CD66+ Enterocyte,Epithelial,Transit Amplifying Zone,Epithelial,Mature Epithelial,Epithelial,CD66+ Mature Epithelial,Epithelial,Mucosa,Mucosa


In [5]:
# Only keep cells from one dataset by selecting 1 donor and 1 region
df_filtered = df[(df['donor'] == "B012") & (
    df['unique_region'] == "B012_Proximal jejunum")]

In [6]:
df_filtered['Cell Type'] = df_filtered['Cell Type'].astype(str) + '_Cell Type'
df_filtered['Neighborhood'] = df_filtered['Neighborhood'].astype(str) + '_Neighborhood'
df_filtered['Community'] = df_filtered['Community'].astype(str) + '_Community'
df_filtered['Tissue Unit'] = df_filtered['Tissue Unit'].astype(str) + '_Tissue Unit'

C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_60572\1316168077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Cell Type'] = df_filtered['Cell Type'].astype(str) + '_Cell Type'
C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_60572\1316168077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Neighborhood'] = df_filtered['Neighborhood'].astype(str) + '_Neighborhood'
C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_60572\1316168077.py:3: SettingWithCopyWarning: 
A value is t

In [7]:
df_filtered

,MUC2,SOX9,MUC1,CD31,Synapto,CD49f,CD15,CHGA,CDX2,ITLN1,...,Cell Type em,Cell subtype,Neighborhood,Neigh_sub,Neighborhood_Ind,NeighInd_sub,Community,Major Community,Tissue Segment,Tissue Unit
1890593,0.216090,1.119012,0.567092,-0.242881,-0.274699,-0.191957,-0.204962,-0.109570,0.029645,0.117278,...,Cycling TA,Epithelial,Transit Amplifying Zone_Neighborhood,Epithelial,Transit Amplifying Zone,Epithelial,Plasma Cell Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
1890594,-0.221731,-0.223280,-0.283894,-0.256254,-0.273986,-0.373476,-0.215366,-0.113153,-0.262340,-0.354025,...,Cycling TA,Epithelial,Plasma Cell Enriched_Neighborhood,Immune,Plasma Cell Enriched,Immune,Plasma Cell Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
1890595,-0.221731,-0.026314,-0.442065,-0.256254,-0.274699,-0.211868,-0.173806,0.101372,-0.019355,-0.153058,...,Cycling TA,Epithelial,Secretory Epithelial_Neighborhood,Epithelial,Adatpive Immune Enriched,Immune,Secretory Epithelial_Community,Epithelial,Mucosa,Mucosa_Tissue Unit
1890596,-0.177174,-0.140475,-0.241711,-0.217204,-0.270773,-0.342078,-0.155135,-0.107398,-0.243269,-0.306894,...,Cycling TA,Epithelial,Macrovasculature_Neighborhood,Stromal,Stroma & Innate Immune,Stromal,Stroma_Community,Stroma,Subucosa,Muscularis externa_Tissue Unit
1890597,-0.166252,-0.155544,-0.225982,-0.227331,-0.255545,-0.349508,-0.211896,-0.107097,-0.221056,-0.295231,...,Cycling TA,Epithelial,Innervated Smooth Muscle_Neighborhood,Stromal,Smooth Muscle,Stromal,Smooth Muscle_Community,Smooth Muscle,Muscle,Muscularis externa_Tissue Unit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056157,-0.221627,-0.223280,-0.361725,-0.233062,-0.255201,-0.379466,-0.215366,-0.113153,-0.284214,-0.296658,...,DC,Immune,Plasma Cell Enriched_Neighborhood,Immune,Plasma Cell Enriched,Immune,Plasma Cell Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
2056158,-0.212460,-0.188303,-0.317383,-0.250673,-0.263856,-0.369020,-0.200340,-0.109447,-0.265941,-0.318743,...,DC,Immune,Innervated Smooth Muscle_Neighborhood,Stromal,Smooth Muscle,Stromal,Smooth Muscle_Community,Smooth Muscle,Muscle,Muscularis externa_Tissue Unit
2056159,-0.218078,-0.207831,-0.338366,-0.256254,-0.249269,-0.379466,-0.212297,-0.106724,-0.266768,-0.295096,...,DC,Immune,Innate Immune Enriched_Neighborhood,Immune,Stroma & Innate Immune,Stromal,Adaptive Immune Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
2056160,-0.203579,-0.219397,-0.307815,0.040683,0.241911,-0.348804,-0.184639,-0.112245,-0.111207,-0.322564,...,DC,Immune,Macrovasculature_Neighborhood,Stromal,Macrovasculature,Stromal,Smooth Muscle_Community,Smooth Muscle,Muscle,Muscularis externa_Tissue Unit


In [8]:
# Next, let's define a function that turns a DataFrame into a node list that can then be passed into the CdeVisualization or NodeDistVis widget
def make_node_list(df:pd.DataFrame, is_3d:bool = False):
  """Turn a DataFrame into a list of dicts for passing them into a HRA widget

  Args:
      df (pd.DataFrame): A DataFrame with cells
  """
  
  # If the df does not have a z-axis column, let's add one and set all cells to 0
  if not is_3d:
    df.loc[:, ('z')] = 0
  
  node_list = [{'x': row['x'], 'y': row['y'], 'z': row['z'], 'level': row['combined_info']}
                 for index, row in df.iterrows()]

  return node_list
  

In [9]:
# # Prepare df_cells for visualization with NodeDistVis widget
# node_list = make_node_list(df_cells, False)

# # Let's inspect the first 5 rows
# pprint(node_list[:5])

# Nodes and edges test

In [10]:
'''
import csv

# Define the path to the CSV file
csv_file_path = '/Users/yang/Desktop/3D_visualization/data/CRC01002-nodes.csv'

# Initialize an empty list to store the nodes
node_list = []

# Open the CSV file
with open(csv_file_path, mode='r') as file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(file)
    
    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Append each row (as a dictionary) to the node_list
        node_list.append(row)
'''

"\nimport csv\n\n# Define the path to the CSV file\ncsv_file_path = '/Users/yang/Desktop/3D_visualization/data/CRC01002-nodes.csv'\n\n# Initialize an empty list to store the nodes\nnode_list = []\n\n# Open the CSV file\nwith open(csv_file_path, mode='r') as file:\n    # Create a CSV reader object\n    csv_reader = csv.DictReader(file)\n    \n    # Iterate over the rows in the CSV file\n    for row in csv_reader:\n        # Append each row (as a dictionary) to the node_list\n        node_list.append(row)\n"

In [11]:
# node_list

In [12]:
'''
import csv

# Define the path to the CSV file
csv_file_path = '/Users/yang/Desktop/3D_visualization/data/CRC01002-edges.csv'

# Initialize an empty list to store the nodes
edge_list = []

# Open the CSV file
with open(csv_file_path, mode='r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Append each row (as a dictionary) to the node_list
        edge_list.append(row)
'''

"\nimport csv\n\n# Define the path to the CSV file\ncsv_file_path = '/Users/yang/Desktop/3D_visualization/data/CRC01002-edges.csv'\n\n# Initialize an empty list to store the nodes\nedge_list = []\n\n# Open the CSV file\nwith open(csv_file_path, mode='r') as file:\n    # Create a CSV reader object\n    csv_reader = csv.reader(file)\n    \n    # Iterate over the rows in the CSV file\n    for row in csv_reader:\n        # Append each row (as a dictionary) to the node_list\n        edge_list.append(row)\n"

In [13]:
# edge_list

In [14]:
'''
node_dist_vis = NodeDistVis(
    nodes=node_list,
    node_target_key="Cell Type",
    node_target_value="Endothelial",
    edges=edge_list,
)
display(node_dist_vis)
'''

'\nnode_dist_vis = NodeDistVis(\n    nodes=node_list,\n    node_target_key="Cell Type",\n    node_target_value="Endothelial",\n    edges=edge_list,\n)\ndisplay(node_dist_vis)\n'

# Display

In [15]:
# # Finally, let's instantiate the NodeDistVis class with some parameters. We pass in the node_list, indicate Endothelial cells as targets for the edges. 
# # As we are not supplying an edge list, we need to provide a max_edge_distance, which is set to 1000 (generiously)
# node_dist_vis = NodeDistVis(
#     nodes = node_list,
#     node_target_key="Cell Type",
#     node_target_value="Endothelial",
#     max_edge_distance = 1000
# )

# # Display our new widget
# display(node_dist_vis)

For comparison, here is Fig. 3 from the paper for comparison with the `NodeDistVis` widget above. Note that this figure shows **cell neighborhoods** rather than **cell types** like the `NodeDistVis`:

<img src="images/fig_3_original.jpg" width="400">

## Next, let's make four regions and visualize a 3D tissue stack.

To showcase the 3D rendering capabilities of our `CDEVisualization` widget, we will display the same section as we showed with the `NodeDistVis` widget above, but we will show it three times with an added offset along the z-axis.

In [16]:
df_filtered

,MUC2,SOX9,MUC1,CD31,Synapto,CD49f,CD15,CHGA,CDX2,ITLN1,...,Cell Type em,Cell subtype,Neighborhood,Neigh_sub,Neighborhood_Ind,NeighInd_sub,Community,Major Community,Tissue Segment,Tissue Unit
1890593,0.216090,1.119012,0.567092,-0.242881,-0.274699,-0.191957,-0.204962,-0.109570,0.029645,0.117278,...,Cycling TA,Epithelial,Transit Amplifying Zone_Neighborhood,Epithelial,Transit Amplifying Zone,Epithelial,Plasma Cell Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
1890594,-0.221731,-0.223280,-0.283894,-0.256254,-0.273986,-0.373476,-0.215366,-0.113153,-0.262340,-0.354025,...,Cycling TA,Epithelial,Plasma Cell Enriched_Neighborhood,Immune,Plasma Cell Enriched,Immune,Plasma Cell Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
1890595,-0.221731,-0.026314,-0.442065,-0.256254,-0.274699,-0.211868,-0.173806,0.101372,-0.019355,-0.153058,...,Cycling TA,Epithelial,Secretory Epithelial_Neighborhood,Epithelial,Adatpive Immune Enriched,Immune,Secretory Epithelial_Community,Epithelial,Mucosa,Mucosa_Tissue Unit
1890596,-0.177174,-0.140475,-0.241711,-0.217204,-0.270773,-0.342078,-0.155135,-0.107398,-0.243269,-0.306894,...,Cycling TA,Epithelial,Macrovasculature_Neighborhood,Stromal,Stroma & Innate Immune,Stromal,Stroma_Community,Stroma,Subucosa,Muscularis externa_Tissue Unit
1890597,-0.166252,-0.155544,-0.225982,-0.227331,-0.255545,-0.349508,-0.211896,-0.107097,-0.221056,-0.295231,...,Cycling TA,Epithelial,Innervated Smooth Muscle_Neighborhood,Stromal,Smooth Muscle,Stromal,Smooth Muscle_Community,Smooth Muscle,Muscle,Muscularis externa_Tissue Unit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056157,-0.221627,-0.223280,-0.361725,-0.233062,-0.255201,-0.379466,-0.215366,-0.113153,-0.284214,-0.296658,...,DC,Immune,Plasma Cell Enriched_Neighborhood,Immune,Plasma Cell Enriched,Immune,Plasma Cell Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
2056158,-0.212460,-0.188303,-0.317383,-0.250673,-0.263856,-0.369020,-0.200340,-0.109447,-0.265941,-0.318743,...,DC,Immune,Innervated Smooth Muscle_Neighborhood,Stromal,Smooth Muscle,Stromal,Smooth Muscle_Community,Smooth Muscle,Muscle,Muscularis externa_Tissue Unit
2056159,-0.218078,-0.207831,-0.338366,-0.256254,-0.249269,-0.379466,-0.212297,-0.106724,-0.266768,-0.295096,...,DC,Immune,Innate Immune Enriched_Neighborhood,Immune,Stroma & Innate Immune,Stromal,Adaptive Immune Enriched_Community,Immune,Mucosa,Mucosa_Tissue Unit
2056160,-0.203579,-0.219397,-0.307815,0.040683,0.241911,-0.348804,-0.184639,-0.112245,-0.111207,-0.322564,...,DC,Immune,Macrovasculature_Neighborhood,Stromal,Macrovasculature,Stromal,Smooth Muscle_Community,Smooth Muscle,Muscle,Muscularis externa_Tissue Unit


In [17]:
# Create a new data frame with values from the NodeDistVis example
df = df_filtered

# indicate the number of layers you would like to show. In our case, let's show 3.
n_layers = 4   # One layer for cell type, one for neighborhood, one for community, one for tissue unit

# Create a list to hold all the data frames
df_list = [df]

for i in range(0, n_layers-1):

  # Create a copy of this new DataFrame
  df_copy = df.copy()

  # Modify a column in the copied rows (e.g., change values in column 'B')
  df_copy['unique_region'] = f'copy_{i}'  

  # Add df_copy to list of df
  df_list.append(df_copy)
  
# Concatenate the original DataFrame with the modified copies
df_combined = pd.concat(df_list, ignore_index=True)

df_filtered_3d = df_combined

In [18]:
# Set a z-offset
offset = 2500

# Set z axis (or any other axis) by region
df_filtered_3d['z'] = df_filtered_3d['unique_region'].apply(lambda v: 0 if v == 'B012_Proximal jejunum'
                                                            else offset if v == 'copy_0'
                                                            else offset * 2 if v == 'copy_1'
                                                            else offset * 3
                                                            )

# Make new df with only x, y, z, Cell Type, and Neighborhood columns
df_cells_3d = df_filtered_3d[['x', 'y', 'z', 'Cell Type', 'Neighborhood', 'Community', 'Tissue Unit']]

cell_type_z0 = df_cells_3d[df_cells_3d['z'] == 0]['Cell Type']
neighborhood_z1 = df_cells_3d[df_cells_3d['z'] == offset]['Neighborhood']
community_z2 = df_cells_3d[df_cells_3d['z'] == offset *2]['Community']
tissueunit_z3 = df_cells_3d[df_cells_3d['z'] == offset *3]['Tissue Unit']

combined_info = pd.concat([cell_type_z0, neighborhood_z1, community_z2, tissueunit_z3])

df_cells_3d['combined_info'] = combined_info


C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_60572\4274359806.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cells_3d['combined_info'] = combined_info


In [19]:
len(df_cells_3d['combined_info'].unique())

58

In [20]:
df_cells_3d

,x,y,z,Cell Type,Neighborhood,Community,Tissue Unit,combined_info
0,7109.0,7160.0,0,Cycling TA_Cell Type,Transit Amplifying Zone_Neighborhood,Plasma Cell Enriched_Community,Mucosa_Tissue Unit,Cycling TA_Cell Type
1,4803.0,55.0,0,Cycling TA_Cell Type,Plasma Cell Enriched_Neighborhood,Plasma Cell Enriched_Community,Mucosa_Tissue Unit,Cycling TA_Cell Type
2,7203.0,7413.0,0,Cycling TA_Cell Type,Secretory Epithelial_Neighborhood,Secretory Epithelial_Community,Mucosa_Tissue Unit,Cycling TA_Cell Type
3,424.0,1437.0,0,Cycling TA_Cell Type,Macrovasculature_Neighborhood,Stroma_Community,Muscularis externa_Tissue Unit,Cycling TA_Cell Type
4,93.0,344.0,0,Cycling TA_Cell Type,Innervated Smooth Muscle_Neighborhood,Smooth Muscle_Community,Muscularis externa_Tissue Unit,Cycling TA_Cell Type
...,...,...,...,...,...,...,...,...
130075,7604.0,6824.0,7500,DC_Cell Type,Plasma Cell Enriched_Neighborhood,Plasma Cell Enriched_Community,Mucosa_Tissue Unit,Mucosa_Tissue Unit
130076,664.0,4597.0,7500,DC_Cell Type,Innervated Smooth Muscle_Neighborhood,Smooth Muscle_Community,Muscularis externa_Tissue Unit,Muscularis externa_Tissue Unit
130077,5695.0,4301.0,7500,DC_Cell Type,Innate Immune Enriched_Neighborhood,Adaptive Immune Enriched_Community,Mucosa_Tissue Unit,Mucosa_Tissue Unit
130078,61.0,3230.0,7500,DC_Cell Type,Macrovasculature_Neighborhood,Smooth Muscle_Community,Muscularis externa_Tissue Unit,Muscularis externa_Tissue Unit


In [21]:
df_cells_3d_filtered = df_cells_3d[['x', 'y', 'z', 'combined_info']]

In [22]:
# Prepare df_cells_3d for visualization with CdeVisualization widget
node_list = make_node_list(df_cells_3d_filtered, True)

# # Let's inspect the first 5 rows
# pprint(node_list[:5])

# for n in node_list:
#     print(n['level'])

# Nodes and edges list construction

In [23]:
# # Filter the DataFrame where the content in the 'combined_info' column is the level name interested
# edge_df = df_cells_3d_filtered[df_cells_3d_filtered['combined_info'] == 'Plasma Cell Enriched_Neighborhood']

# # Select the 'x', 'y', and 'z' columns from the filtered DataFrame
# edge_df_filtered = edge_df[['x', 'y', 'z', 'combined_info']]


In [24]:
# # Extract the corresponding information in the cell types
# edge_index = edge_df_filtered.index - len(df_filtered)
# edge_df_2 = df_cells_3d_filtered.iloc[edge_index]

In [25]:
# edge_df_filtered = edge_df[['x', 'y', 'z']]
# edge_df_2_filtered = edge_df_2[['x', 'y', 'z']]
# edge_df_filtered = edge_df_filtered.reset_index(drop=True)
# edge_df_2_filtered = edge_df_2_filtered.reset_index(drop=True)

In [26]:
# # Combine the selected columns horizontally
# edge_list_df = pd.concat([edge_df_filtered, edge_df_2_filtered], axis=1)

# edge_list_df = edge_list_df.reset_index(drop=False)

# # Convert the DataFrame to a list of lists with each value as a string
# edge_list = edge_list_df.astype(str).values.tolist()

In [27]:
# edge_list

# Display

In [28]:
edge_list = ['', '', '', '', '', '', '']

In [29]:
# Finally, let's instantiate the CDEVisualization class with our node_list as parameter.
cde = CdeVisualization(
    nodes=node_list,
    node_target_key = "level",
    #node_target_value = "Enterocyte_Cell Type",
    edges=edge_list,
)

# Display our new widget
display(cde)

CdeVisualization(age=None, color_map=None, color_map_key=None, color_map_value_key=None, creation_timestamp=No…